# CIS600 - Social Media & Data Mining
###  
<img src="https://www.syracuse.edu/wp-content/themes/g6-carbon/img/syracuse-university-seal.svg?ver=6.3.9" style="width: 200px;"/>

### February 6, 2018

## Community Detection

In [1]:
import numpy as np
import networkx as nx
# Note changes; we need tools for manipulating labels
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models.annotations import LabelSet
from bokeh.models.graphs import from_networkx

In [2]:
# Let's define our favorite graph
A = np.array([[0,1,1,1,0,0,0,0,0],[1,0,1,0,0,0,0,0,0],[1,1,0,1,0,0,0,0,0],[1,0,1,0,1,1,0,0,0],[0,0,0,1,0,1,1,1,0],
             [0,0,0,1,1,0,1,1,0],[0,0,0,0,1,1,0,1,1],[0,0,0,0,1,1,1,0,0],[0,0,0,0,0,0,1,0,0]])
H = nx.from_numpy_matrix(A)

### Cliques

#### We can identify the cliques of H by visual inspection, or we can ask networkx for them. In the cell below, we find some cliques.

In [4]:
cliques_generator = nx.find_cliques(H)
list(cliques_generator)

[[1, 0, 2], [3, 0, 2], [3, 4, 5], [6, 8], [6, 4, 5, 7]]

#### Notice that the cliques are overlapping, but each one is *maximal*. Why? Also, notice that there is exactly one *maximum* clique. This function computes all maximal cliques (see docs).

In [5]:
help(nx.find_cliques)

Help on function find_cliques in module networkx.algorithms.clique:

find_cliques(G)
    Returns all maximal cliques in an undirected graph.
    
    For each node *v*, a *maximal clique for v* is a largest complete
    subgraph containing *v*. The largest maximal clique is sometimes
    called the *maximum clique*.
    
    This function returns an iterator over cliques, each of which is a
    list of nodes. It is an iterative implementation, so should not
    suffer from recursion depth issues.
    
    Parameters
    ----------
    G : NetworkX graph
        An undirected graph.
    
    Returns
    -------
    iterator
        An iterator over maximal cliques, each of which is a list of
        nodes in `G`. The order of cliques is arbitrary.
    
    See Also
    --------
    find_cliques_recursive
        A recursive version of the same algorithm.
    
    Notes
    -----
    To obtain a list of all maximal cliques, use
    `list(find_cliques(G))`. However, be aware that in the w

#### We can also ask for *all* of the cliques.

In [6]:
all_cliques = nx.enumerate_all_cliques(H)
cliques_list = list(all_cliques)
cliques_list

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [0, 1],
 [0, 2],
 [0, 3],
 [1, 2],
 [2, 3],
 [3, 4],
 [3, 5],
 [4, 5],
 [4, 6],
 [4, 7],
 [5, 6],
 [5, 7],
 [6, 7],
 [6, 8],
 [0, 1, 2],
 [0, 2, 3],
 [3, 4, 5],
 [4, 5, 6],
 [4, 5, 7],
 [4, 6, 7],
 [5, 6, 7],
 [4, 5, 6, 7]]

#### There are many more of these! Some of them must fail the condition of maximality. Example?

### $k$-Cliques

#### We can select the size of cliques we want. For example, we can find all $k$-cliques for $k = 3$

In [7]:
k_clique_3 = [x for x in cliques_list if len(x) == 3]

In [21]:
print(k_clique_3)

[[0, 1, 2], [0, 2, 3], [3, 4, 5], [4, 5, 6], [4, 5, 7], [4, 6, 7], [5, 6, 7]]


#### Can you find a function in networkx that already does this? Can you think of a better way?

#### We want the $k$-clique communities. We are discovering structure in H and we must do some computation. Recall that we now think of cliques themselves being adjacent, then look at the resulting edges.

In [8]:
help(nx.community.k_clique_communities)

Help on function k_clique_communities in module networkx.algorithms.community.kclique:

k_clique_communities(G, k, cliques=None)
    Find k-clique communities in graph using the percolation method.
    
    A k-clique community is the union of all cliques of size k that
    can be reached through adjacent (sharing k-1 nodes) k-cliques.
    
    Parameters
    ----------
    G : NetworkX graph
    
    k : int
       Size of smallest clique
    
    cliques: list or generator       
       Precomputed cliques (use networkx.find_cliques(G))
    
    Returns
    -------
    Yields sets of nodes, one for each k-clique community.
    
    Examples
    --------
    >>> from networkx.algorithms.community import k_clique_communities
    >>> G = nx.complete_graph(5)
    >>> K5 = nx.convert_node_labels_to_integers(G,first_label=2)
    >>> G.add_edges_from(K5.edges())
    >>> c = list(k_clique_communities(G, 4))
    >>> list(c[0])
    [0, 1, 2, 3, 4, 5, 6]
    >>> list(k_clique_communities(G, 6))

In [22]:
k_comm_3 = list(nx.community.k_clique_communities(H,3))
k_comm_3

[frozenset({0, 1, 2, 3}), frozenset({3, 4, 5, 6, 7})]

#### What is this "frozenset" business?

### Clique Percolation

#### The k_clique_communities function implements CPM. Let's try that and see what we get.

In [12]:
# First, let's find the 'edges' in the new graph
comm_edges = []
K = len(k_clique_3)
for u in range(K):
    for v in range(u):
        U = set(k_clique_3[u])
        V = set(k_clique_3[v])
        if len(U.intersection(V)) == 2:
            comm_edges.append((u,v))

In [23]:
# Next, let's build a new graph
HC = nx.Graph()
# And give it nodes...
HC.add_nodes_from(range(K))
# And the edges we calculated...
HC.add_edges_from(comm_edges)

In [24]:
# Finally, we use another networkx function to find the components.
comps_generator = nx.components.connected_components(HC)

In [25]:
list(comps_generator)

[{0, 1}, {2, 3, 4, 5, 6}]

#### Let's think carefully about this; these are *not* nodes from the original graph H! These are indices of elements of the list of $k$-cliques. Look at the first set in the above list.

In [19]:
x = [set(c) for c in k_clique_3[:2]]
set.union(*x)

{0, 1, 2, 3}

#### The union of those nodes, from the original H, is in fact $\{0,1,2,3\}$, just as calculated by networkx. How about the other $k$-community?

In [26]:
x = [set(c) for c in k_clique_3[2:]]
set.union(*x)

{3, 4, 5, 6, 7}

#### Indeed, there are the other four nodes. Notice that the end-node $9$ is left out of both communities - not surprising.

#### This shows how to add labels to nodes in the appropriate positions.

In [27]:
# The positions must be gotten from the bokeh object "graph"
output_notebook()

plot = figure(title="Networkx Integration Demonstration", x_range=(-3,3), y_range=(-3,3),
              tools="", toolbar_location=None)

graph = from_networkx(H, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)


pos = graph.layout_provider.graph_layout
x,y=zip(*pos.values())

source = ColumnDataSource({'x':x,'y':y,'kid':['V%d' %ix for ix in range(len(x))]})
labels = LabelSet(x='x', y='y', text='kid', source=source)

plot.renderers.append(labels)


show(plot)

Loading BokehJS ...